



# **IP2_Run3**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))



### **Generate Mathematical Model**

In [ ]:
problem = """Your goal is to invest in several of 10 possible investment strategies in the most optimal way. The historic returns of those strategies are stored in the file "investments_data.csv". Each column represents one strategy and the rows are the past investment outcomes. There is no index and the values are separated by a ;.

The costs for investing in a given investment is stored in a vector A, which has one value for each strategy in order.
The values are: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]

You can only invest once into an investment.

Unfortunately due to other costs and inflation, your available budget at this time is uncertain. There are four possible budget scenarios with different probabilities: scenario 1 with 1000 euros and probability of 0.55, scenario 2  with 1100 euros and probability of 0.4, scenario 3 with 900 euros and probability of 0.04, scenario 4 with 1200 euros and probability of 0.01.
The tolerable probability of exceeding the budget is 0.4.

Please formulate a mean-variance mathematical model for this optimization problem, considering the past performance of investment strategies and the uncertain budget. You can take 2 as the risk parameter r. """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [ ]:
print(response1.choices[0].message.content)

Parameters:
- µi = expected return of investment i, calculated from the data in "investments_data.csv"
- σi = standard deviation of return of investment i, calculated from the data in "investments_data.csv"
- ai = cost of investing in investment i, taken from G = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]
- bi = budget available in scenario i, i ∈ {1,2,3,4}
- pi = probability of scenario i, i ∈ {1,2,3,4}
- r = risk parameter, given as 2

Decision variables:
- xi ∈ {0, 1}, the decision to invest (1) or not invest (0) in investment i

Objective function:
Minimize cost and risk while maximizing return. The objective function is subject to the risk parameter r.

min U = Σ xi*(r*σi - µi)

subject to the following constraints:

1. Budget constraints:

Σ ai*xi ≤ bi, for each scenario i, i ∈ {1,2,3,4}
Σ pi*(Σ ai*xi > bi) ≤ 0.4, where "Σ ai*xi > bi" yields 1 when true, else 0.

2. Investment constraint:
xi ∈ {0, 1}, for all investment i

This model aims at selecting the combination of inve

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here is the python pyomo code for this optimization problem:

```python
from pyomo.environ import *

# These are indices and sample data
I = range(10)
μ = [random.random() for i in I] # We are using random numbers for expected returns of investments for the sake of example
σ = [random.random() for i in I] # Same for standard deviation
G = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225] # Cost for investment
r = 2 # Risk parameter 
Sc = [1,2,3,4] # Scenarios
b = [1000, 2000, 3000, 4000] # Budgets for scenarios
p = [0.25,0.25,0.25,0.25] # Probabilities for Scenarios, assuming equal probability for illustrative purposes. 

model = ConcreteModel()

# Decision variable
model.x = Var(I, within=Binary)

# Objective function
model.obj = Objective(expr = sum(model.x[i] * (r*σ[i] - μ[i]) for i in I), sense=minimize)

# Budget constraints for the scenarios
model.budget = ConstraintList()
for s in Sc:
    model.budget.add(sum(model.x[i]*G[i] for i in I) <= b[s])

# Constraint for the total probability of exceeding the budget
model.prob = Constraint(expr = sum(p[s]*(sum(model.x[i]*G[i] for i in I) > b[s]) for s in Sc) <= 0.4)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the solution: which investment i to include
print('Solution:')
for i in I:
    print('Investment ', i, ': ', model.x[i].value)
```

This code will give you which investments to take. Note that we used random values for expected returns and standard deviations. To achieve optimum results, it's better to use real-world data. Besides, the scenario probabilities here are assumed to be equal for simplicity, which might not hold in a real-world scenario. Replace these values with your own data. 

This code assumes that you have GLPK solver installed on your machine.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *
import random
# These are indices and sample data
I = range(10)
μ = [random.random() for i in I] # We are using random numbers for expected returns of investments for the sake of example
σ = [random.random() for i in I] # Same for standard deviation
G = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225] # Cost for investment
r = 2 # Risk parameter
Sc = [1,2,3,4] # Scenarios
b = [1000, 2000, 3000, 4000] # Budgets for scenarios
p = [0.25,0.25,0.25,0.25] # Probabilities for Scenarios, assuming equal probability for illustrative purposes.

model = ConcreteModel()

# Decision variable
model.x = Var(I, within=Binary)

# Objective function
model.obj = Objective(expr = sum(model.x[i] * (r*σ[i] - μ[i]) for i in I), sense=minimize)

# Budget constraints for the scenarios
model.budget = ConstraintList()
for s in Sc:
    model.budget.add(sum(model.x[i]*G[i] for i in I) <= b[s])

# Constraint for the total probability of exceeding the budget
model.prob = Constraint(expr = sum(p[s]*(sum(model.x[i]*G[i] for i in I) > b[s]) for s in Sc) <= 0.4)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the solution: which investment i to include
print('Solution:')
for i in I:
    print('Investment ', i, ': ', model.x[i].value)

IndexError: list index out of range

In [ ]:
from pyomo.environ import *
import random
# These are indices and sample data
I = range(10)
#start - data inputted by human
df = pd.read_csv("investments_data.csv",sep=";", header=None)
headers = []
for i in range(len(df.columns)):
    headers.append("Investment"+str((i+1)))
df.columns = headers

μ = df.mean(0)
σ = df.var(0)
#end
G = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225] # Cost for investment
r = 2 # Risk parameter
Sc = [1,2,3,4] # Scenarios
b = [1000, 1100, 900, 1200] # Budgets for scenarios
p = [0.55,0.4,0.04,0.01] # Probabilities for Scenarios, assuming equal probability for illustrative purposes.

model = ConcreteModel()

# Decision variable
model.x = Var(I, within=Binary)

# Objective function
model.obj = Objective(expr = sum(model.x[i] * (r*σ[i] - μ[i]) for i in I), sense=minimize)

# Budget constraints for the scenarios
model.budget = ConstraintList()
for s in Sc:
    model.budget.add(sum(model.x[i]*G[i] for i in I) <= b[s])

# Constraint for the total probability of exceeding the budget
model.prob = Constraint(expr = sum(p[s]*(sum(model.x[i]*G[i] for i in I) > b[s]) for s in Sc) <= 0.4)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the solution: which investment i to include
print('Solution:')
for i in I:
    print('Investment ', i, ': ', model.x[i].value)

IndexError: list index out of range

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [ ]:
from pyomo.environ import *
import pandas as pd
# These are indices and sample data
I = range(10)

#start - data inputted by human
df = pd.read_csv("investments_data.csv",sep=";", header=None)
headers = []
for i in range(len(df.columns)):
    headers.append("Investment"+str((i+1)))
df.columns = headers

μ = df.mean(0)
σ = df.var(0)
#end
G = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225] # Cost for investment
r = 2 # Risk parameter
Sc = range(4) # Scenarios
b = [1000, 1100, 900, 1200] # Budgets for scenarios
p = [0.55,0.4,0.04,0.01] # Probabilities for Scenarios, assuming equal probability for illustrative purposes.
M = sum(G)

model = ConcreteModel()

# Decision variable
model.x = Var(I, within=Binary)

# Objective function
model.obj = Objective(expr = sum(model.x[i] * (r*σ[i] - μ[i]) for i in I), sense=minimize)
# Budget constraints for the scenarios
model.budget = ConstraintList()

model.bin = Var(Sc, within=Binary)
for s in Sc:
    # Budget constraint
    model.budget.add(sum(model.x[i] * G[i] for i in I) <= b[s])
    # Constraint to link the binary variable
    model.budget.add(sum(model.x[i] * G[i] for i in I) - b[s] <= M * model.bin[s])
    model.budget.add(sum(model.x[i] * G[i] for i in I) - b[s] <= M * (1 - model.bin[s]))

model.prob = Constraint(expr=sum(p[s] * model.bin[s] for s in Sc) <= 0.4)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the solution: which investment i to include
print('Solution:')
for i in I:
    print('Investment ', i+1, ': ', model.x[i].value)
print(model.obj())

Solution:
Investment  1 :  0.0
Investment  2 :  0.0
Investment  3 :  1.0
Investment  4 :  0.0
Investment  5 :  0.0
Investment  6 :  1.0
Investment  7 :  0.0
Investment  8 :  0.0
Investment  9 :  0.0
Investment  10 :  1.0
-188.47780903195851
